In [135]:
import requests
import pandas as pd
import json
import sys
sys.path.append('../')
from dateutil import parser
from infrastructure.instrument_collection import instrumentCollection
# from datetime import datetime as datetime
from datetime import timedelta, datetime
pd.set_option('display.max_columns', None)
pd.set_option('expand_frame_repr', False)

import pandas as pd
import sys
from technicals.patterns import apply_patterns
from models.trade_decision import TradeDecision

from constants import defs
from technicals.indicators import BollingerBands
sys.path.append("../")
from models.tradesettings import TradeSettings

In [136]:
API_KEY = "69f109a1f4f5cc5df743768870afa57d-7eeba7f9b30b063c75a4c3e6fceeb083"
ACCOUNT_ID = "101-004-29766214-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3" 

In [137]:
session = requests.Session()

In [138]:
session.headers.update({"Authorization": f"Bearer {API_KEY}", "Content-Type":"application/json"})

In [139]:
params = dict(count=10, price="MBA",granularity="H1" )

In [168]:
instrument = "EUR_JPY"
granularity = "M1"


In [169]:
fileName = f"my_candles{instrument}_{granularity}.json"
path  = "../data"
full_path = f'{path}/{fileName}'

In [170]:
new_df = pd.read_json(full_path)

In [171]:
df_copy  = new_df.copy() 

In [172]:
# mas =  [10,20,50,100,200]
# for i in mas:
#     df_an[f'MA_{i}']= df_an['mid_o'].rolling(window=i).mean()
# df_an.dropna(inplace=True)

In [173]:
df_copy

,time,mid_o,mid_h,mid_l,mid_c,ask_o,ask_h,ask_l,ask_c,bid_o,bid_h,bid_l,bid_c
0,2024-01-01T22:03:00.000000000Z,155.628,155.628,155.628,155.628,155.703,155.703,155.703,155.703,155.553,155.553,155.553,155.553
1,2024-01-01T22:05:00.000000000Z,155.659,155.659,155.659,155.659,155.734,155.734,155.734,155.734,155.584,155.584,155.584,155.584
2,2024-01-01T22:07:00.000000000Z,155.618,155.618,155.608,155.608,155.693,155.693,155.683,155.683,155.543,155.543,155.533,155.533
3,2024-01-01T22:09:00.000000000Z,155.610,155.610,155.610,155.610,155.682,155.682,155.682,155.682,155.539,155.539,155.539,155.539
4,2024-01-01T22:10:00.000000000Z,155.610,155.614,155.610,155.613,155.681,155.684,155.673,155.673,155.540,155.553,155.540,155.553
...,...,...,...,...,...,...,...,...,...,...,...,...,...
256248,2024-09-06T20:55:00.000000000Z,157.780,157.780,157.722,157.722,157.802,157.805,157.744,157.748,157.759,157.759,157.697,157.697
256249,2024-09-06T20:56:00.000000000Z,157.724,157.762,157.722,157.732,157.749,157.801,157.743,157.778,157.700,157.738,157.687,157.687
256250,2024-09-06T20:57:00.000000000Z,157.736,157.776,157.728,157.768,157.777,157.822,157.766,157.815,157.694,157.731,157.685,157.721
256251,2024-09-06T20:58:00.000000000Z,157.770,157.770,157.742,157.752,157.819,157.822,157.788,157.802,157.721,157.721,157.687,157.703


In [174]:
def apply_SL(row,trade_settings: TradeSettings ):
    if row.SIGNAL == defs.BUY:
        return row.mid_c - (row.GAIN)
    if row.SIGNAL == defs.SELL:
        return row.mid_c + (row.GAIN)
    return 0.0
    # if row.SIGNAL == defs.BUY:
    #     return row.mid_c - (row.GAIN/trade_settings.riskreward)
    # if row.SIGNAL == defs.SELL:
    #     return row.mid_c + (row.GAIN/trade_settings.riskreward)
    # return 0.0

def apply_TP(row):
    if row.SIGNAL == defs.BUY:
        return row.mid_c + row.GAIN
    if row.SIGNAL == defs.SELL:
        return row.mid_c - row.GAIN
    return 0.0
# def apply_signal(row, tradeSettings:TradeSettings):
#     if row.SPREAD <= tradeSettings.maxspread and row.GAIN >= tradeSettings.mingain:
#         if row.PREV_CROSS_BB_UP == 1 and row.mid_c < row.BB_UP:
#             return defs.SELL
#         elif row.mid_c < row.BB_LW and row.mid_o > row.BB_LW:
#             return defs.BUY
#     return defs.NONE
def apply_cross_boll_up(row):
    if row.mid_c > row.BB_UP : #and row.mid_o < row.BB_UP:
            return 1
    else:
        return 0
def apply_signal(row, tradeSettings:TradeSettings):
    if row.SPREAD <= tradeSettings.maxspread and row.GAIN >= tradeSettings.mingain:
        if row.mid_c > row.BB_UP and row.mid_o < row.BB_UP:
            return defs.SELL
        elif row.mid_c < row.BB_LW and row.mid_o > row.BB_LW:
            return defs.BUY
    return defs.NONE


In [175]:
def process_candles(df: pd.DataFrame, pair, trade_settings: TradeSettings):
    df_an = df.copy()
    df_an.reset_index(drop=True, inplace=True)
    df_an["PAIR"] = pair
    df_an["SPREAD"] = df_an.ask_c -df_an.bid_c
    df_an = BollingerBands(df_an,trade_settings.n_ma,trade_settings.n_std)
    df_an["GAIN"] = (df_an.mid_c - df_an.BB_MA)*0.6
    df_an["CROSS_BB_UP"] = df_an.apply(apply_cross_boll_up, axis=1)
    df_an["PREV_CROSS_BB_UP"] = df_an["CROSS_BB_UP"].shift(1)
    df_an['SIGNAL'] = df_an.apply(apply_signal, axis=1,  args=(trade_settings,))
    df_an["SL"] = df_an.apply(apply_SL, axis=1, args=(trade_settings,))
    df_an["TP"] = df_an.apply(apply_TP,axis=1)
    df_an["LOSS"] = abs(df_an.mid_c - df_an.SL)

    log_cols = ['PAIR', 'time', 'mid_c', 'mid_o','BB_UP' ,'BB_MA','BB_LW','SL', 'TP', 'SPREAD', 'GAIN', 'LOSS', 'SIGNAL']

    # log_cols = ['PAIR', 'time', 'mid_c', 'mid_o', 'SL', 'TP', 'SPREAD', 'GAIN', 'LOSS', 'SIGNAL']
    print(f"process_candles:\n{df_an[log_cols].tail()}", pair)

    return  df_an  #df_an[log_cols].iloc[-1]

In [176]:
settings = ""
tradeSettings  = {}
trade_risk  = 0
granularity  = "M1"

In [177]:
with open("../bot/settings.json","r") as f:
            settings = json.loads(f.read())
            tradeSettings  = {k:TradeSettings(k,v) for k,v in settings['trading_pairs'].items()}
            trade_risk  = settings['trade_risk']
            granularity  = settings['granularity']

In [178]:
tradeSettings

{'EUR_USD': {'pair': 'EUR_USD', 'n_ma': 10, 'n_std': 2.0, 'maxspread': 0.0004, 'mingain': 0.0006, 'riskreward': 1.5},
 'GBP_JPY': {'pair': 'GBP_JPY', 'n_ma': 10, 'n_std': 2.0, 'maxspread': 0.04, 'mingain': 0.06, 'riskreward': 1.5},
 'EUR_JPY': {'pair': 'EUR_JPY', 'n_ma': 10, 'n_std': 2.0, 'maxspread': 0.04, 'mingain': 0.06, 'riskreward': 1.5},
 'USD_JPY': {'pair': 'USD_JPY', 'n_ma': 10, 'n_std': 2.0, 'maxspread': 0.04, 'mingain': 0.06, 'riskreward': 1.5},
 'GBP_USD': {'pair': 'GBP_USD', 'n_ma': 10, 'n_std': 2.0, 'maxspread': 0.0004, 'mingain': 0.0006, 'riskreward': 1.5},
 'EUR_GBP': {'pair': 'EUR_GBP', 'n_ma': 10, 'n_std': 2.0, 'maxspread': 0.0004, 'mingain': 0.0006, 'riskreward': 1.5},
 'USD_CAD': {'pair': 'USD_CAD', 'n_ma': 10, 'n_std': 2.0, 'maxspread': 0.0004, 'mingain': 0.0006, 'riskreward': 1.5}}

In [179]:
df_an = process_candles(df_copy, instrument, tradeSettings[instrument])

process_candles:
           PAIR                            time    mid_c    mid_o       BB_UP       BB_MA       BB_LW   SL   TP  SPREAD     GAIN     LOSS  SIGNAL
256248  EUR_JPY  2024-09-06T20:55:00.000000000Z  157.722  157.780  157.790789  157.753433  157.716077  0.0  0.0   0.051 -0.01886  157.722       0
256249  EUR_JPY  2024-09-06T20:56:00.000000000Z  157.732  157.724  157.787363  157.755100  157.722837  0.0  0.0   0.091 -0.01386  157.732       0
256250  EUR_JPY  2024-09-06T20:57:00.000000000Z  157.768  157.736  157.787866  157.756233  157.724601  0.0  0.0   0.094  0.00706  157.768       0
256251  EUR_JPY  2024-09-06T20:58:00.000000000Z  157.752  157.770  157.788060  157.756767  157.725473  0.0  0.0   0.099 -0.00286  157.752       0
256252  EUR_JPY  2024-09-06T20:59:00.000000000Z  157.754  157.754  157.786419  157.758500  157.730581  0.0  0.0   0.096 -0.00270  157.754       0 EUR_JPY


In [180]:
df_an

,time,mid_o,mid_h,mid_l,mid_c,ask_o,ask_h,ask_l,ask_c,bid_o,bid_h,bid_l,bid_c,PAIR,SPREAD,BB_MA,BB_UP,BB_LW,GAIN,CROSS_BB_UP,PREV_CROSS_BB_UP,SIGNAL,SL,TP,LOSS
0,2024-01-01T22:03:00.000000000Z,155.628,155.628,155.628,155.628,155.703,155.703,155.703,155.703,155.553,155.553,155.553,155.553,EUR_JPY,0.150,NaN,NaN,NaN,NaN,0,NaN,0,0.0,0.0,155.628
1,2024-01-01T22:05:00.000000000Z,155.659,155.659,155.659,155.659,155.734,155.734,155.734,155.734,155.584,155.584,155.584,155.584,EUR_JPY,0.150,NaN,NaN,NaN,NaN,0,0.0,0,0.0,0.0,155.659
2,2024-01-01T22:07:00.000000000Z,155.618,155.618,155.608,155.608,155.693,155.693,155.683,155.683,155.543,155.543,155.533,155.533,EUR_JPY,0.150,NaN,NaN,NaN,NaN,0,0.0,0,0.0,0.0,155.608
3,2024-01-01T22:09:00.000000000Z,155.610,155.610,155.610,155.610,155.682,155.682,155.682,155.682,155.539,155.539,155.539,155.539,EUR_JPY,0.143,NaN,NaN,NaN,NaN,0,0.0,0,0.0,0.0,155.610
4,2024-01-01T22:10:00.000000000Z,155.610,155.614,155.610,155.613,155.681,155.684,155.673,155.673,155.540,155.553,155.540,155.553,EUR_JPY,0.120,NaN,NaN,NaN,NaN,0,0.0,0,0.0,0.0,155.613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256248,2024-09-06T20:55:00.000000000Z,157.780,157.780,157.722,157.722,157.802,157.805,157.744,157.748,157.759,157.759,157.697,157.697,EUR_JPY,0.051,157.753433,157.790789,157.716077,-0.01886,0,0.0,0,0.0,0.0,157.722
256249,2024-09-06T20:56:00.000000000Z,157.724,157.762,157.722,157.732,157.749,157.801,157.743,157.778,157.700,157.738,157.687,157.687,EUR_JPY,0.091,157.755100,157.787363,157.722837,-0.01386,0,0.0,0,0.0,0.0,157.732
256250,2024-09-06T20:57:00.000000000Z,157.736,157.776,157.728,157.768,157.777,157.822,157.766,157.815,157.694,157.731,157.685,157.721,EUR_JPY,0.094,157.756233,157.787866,157.724601,0.00706,0,0.0,0,0.0,0.0,157.768
256251,2024-09-06T20:58:00.000000000Z,157.770,157.770,157.742,157.752,157.819,157.822,157.788,157.802,157.721,157.721,157.687,157.703,EUR_JPY,0.099,157.756767,157.788060,157.725473,-0.00286,0,0.0,0,0.0,0.0,157.752


In [181]:
df_an.loc[df_an.SIGNAL != 0]    #.iloc[-50:]

,time,mid_o,mid_h,mid_l,mid_c,ask_o,ask_h,ask_l,ask_c,bid_o,bid_h,bid_l,bid_c,PAIR,SPREAD,BB_MA,BB_UP,BB_LW,GAIN,CROSS_BB_UP,PREV_CROSS_BB_UP,SIGNAL,SL,TP,LOSS
100,2024-01-01T23:48:00.000000000Z,155.638,155.730,155.638,155.728,155.651,155.744,155.651,155.742,155.625,155.717,155.625,155.715,EUR_JPY,0.027,155.610133,155.677233,155.543034,0.07072,1,1.0,-1,155.79872,155.65728,0.07072
265,2024-01-02T02:33:00.000000000Z,156.034,156.194,156.022,156.142,156.044,156.203,156.031,156.149,156.023,156.184,156.012,156.134,EUR_JPY,0.015,155.953967,156.090401,155.817532,0.11282,1,1.0,-1,156.25482,156.02918,0.11282
730,2024-01-02T10:18:00.000000000Z,155.900,155.965,155.896,155.965,155.910,155.976,155.905,155.976,155.890,155.954,155.887,155.954,EUR_JPY,0.022,155.861833,155.940986,155.782681,0.06190,1,0.0,-1,156.02690,155.90310,0.06190
862,2024-01-02T12:30:00.000000000Z,155.778,155.880,155.775,155.878,155.787,155.889,155.785,155.888,155.769,155.870,155.765,155.869,EUR_JPY,0.019,155.771833,155.831899,155.711767,0.06370,1,0.0,-1,155.94170,155.81430,0.06370
1052,2024-01-02T15:40:00.000000000Z,155.382,155.458,155.382,155.458,155.392,155.469,155.392,155.469,155.372,155.447,155.372,155.447,EUR_JPY,0.022,155.322700,155.434000,155.211400,0.08118,1,0.0,-1,155.53918,155.37682,0.08118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255948,2024-09-06T15:55:00.000000000Z,157.648,157.744,157.646,157.741,157.659,157.756,157.656,157.751,157.637,157.734,157.636,157.731,EUR_JPY,0.020,157.573567,157.692435,157.454698,0.10046,1,1.0,-1,157.84146,157.64054,0.10046
255978,2024-09-06T16:25:00.000000000Z,157.940,157.984,157.924,157.978,157.952,157.993,157.933,157.989,157.928,157.977,157.914,157.967,EUR_JPY,0.022,157.868167,157.963928,157.772405,0.06590,1,1.0,-1,158.04390,157.91210,0.06590
255979,2024-09-06T16:26:00.000000000Z,157.975,158.034,157.964,158.024,157.985,158.044,157.974,158.033,157.965,158.025,157.954,158.016,EUR_JPY,0.017,157.884833,158.012141,157.757525,0.08350,1,1.0,-1,158.10750,157.94050,0.08350
256014,2024-09-06T17:01:00.000000000Z,157.770,157.880,157.769,157.874,157.779,157.893,157.777,157.884,157.760,157.870,157.760,157.865,EUR_JPY,0.019,157.750467,157.842043,157.658890,0.07412,1,0.0,-1,157.94812,157.79988,0.07412


In [182]:
open_trades = []
lost_trades = []
profitable_trades = []
BUY=1
SELL = -1
NONE = 0

In [183]:
def update(idx,trade_decision, row,lost_trades,profitable_trades):
        if trade_decision.signal == BUY:
            if row.bid_h >= trade_decision.tp:
                profitable_trades.append(trade_decision)
                del open_trades[idx]
            elif row.bid_l <= trade_decision.tp:
                lost_trades.append(trade_decision)
                del open_trades[idx]
        if trade_decision.signal == SELL:
            if row.ask_l <= trade_decision.tp:
                profitable_trades.append(trade_decision)
                del open_trades[idx]
            elif row.ask_h >= trade_decision.sl:
                lost_trades.append(trade_decision)
                del open_trades[idx]

In [184]:

for index, row in df_an.iterrows():
    
    if row.SIGNAL != 0:
        open_trades.append(TradeDecision(row,index))  
        
    for idx,ot in enumerate(open_trades):
        update(idx,ot, row,lost_trades,profitable_trades)
       

In [185]:
len(lost_trades)


481

In [186]:
loss=0
for i in lost_trades:
    loss += i.loss
loss

42.79175999999981

In [187]:
len(profitable_trades)

839

In [188]:
profit=0
for i in profitable_trades:
    profit += i.gain
profit

79.80064000000003

In [198]:
# 0.01390072727272704
# 0.014326399999999849 ...10
# 0.012350888888888531  ...9
 
reward = profit-loss
efficiency = len(profitable_trades)/(len(lost_trades)+len(profitable_trades))
win_loss = len(profitable_trades)/(len(lost_trades))
reward ,instrument,df_copy.iloc[1].time,df_copy.iloc[-1].time,efficiency,win_loss

(37.00888000000022,
 'EUR_JPY',
 '2024-01-01T22:05:00.000000000Z',
 '2024-09-06T20:59:00.000000000Z',
 0.6356060606060606,
 1.7442827442827442)

In [190]:
# 0.005422944444444289 ...12
# 0.012095066666666597 ...10
# 0.011189999999999764 ...9
# 0.8030999999999011
# 0.7899833333334421
len(open_trades)

0

In [191]:
for i in lost_trades:
    print(i)

TradeDecision():2467 EUR_JPY dir:-1 gain:0.0671 loss:0.0671 sl:156.5431  tp:156.4089
TradeDecision():3013 EUR_JPY dir:-1 gain:0.0635 loss:0.0635 sl:156.4145  tp:156.2875
TradeDecision():3476 EUR_JPY dir:-1 gain:0.0813 loss:0.0813 sl:157.1593  tp:156.9967
TradeDecision():3477 EUR_JPY dir:-1 gain:0.0930 loss:0.0930 sl:157.2170  tp:157.0310
TradeDecision():3516 EUR_JPY dir:-1 gain:0.0847 loss:0.0847 sl:157.3487  tp:157.1793
TradeDecision():3582 EUR_JPY dir:-1 gain:0.0809 loss:0.0809 sl:157.9789  tp:157.8171
TradeDecision():4799 EUR_JPY dir:-1 gain:0.0679 loss:0.0679 sl:158.6599  tp:158.5241
TradeDecision():4976 EUR_JPY dir:-1 gain:0.0746 loss:0.0746 sl:158.5506  tp:158.4014
TradeDecision():7734 EUR_JPY dir:-1 gain:0.0676 loss:0.0676 sl:157.9036  tp:157.7684
TradeDecision():8215 EUR_JPY dir:-1 gain:0.1177 loss:0.1177 sl:157.6997  tp:157.4643
TradeDecision():8784 EUR_JPY dir:-1 gain:0.0737 loss:0.0737 sl:158.2497  tp:158.1023
TradeDecision():9631 EUR_JPY dir:-1 gain:0.0730 loss:0.0730 sl:15

In [192]:
index = 6885 

df_pl =df_an.iloc[index-30:index+50].copy()
df_pl

,time,mid_o,mid_h,mid_l,mid_c,ask_o,ask_h,ask_l,ask_c,bid_o,bid_h,bid_l,bid_c,PAIR,SPREAD,BB_MA,BB_UP,BB_LW,GAIN,CROSS_BB_UP,PREV_CROSS_BB_UP,SIGNAL,SL,TP,LOSS
6855,2024-01-08T16:50:00.000000000Z,157.860,157.871,157.858,157.859,157.870,157.881,157.867,157.869,157.851,157.861,157.849,157.849,EUR_JPY,0.020,157.864333,157.878756,157.849911,-0.00320,0,0.0,0,0.0,0.0,157.859
6856,2024-01-08T16:51:00.000000000Z,157.856,157.884,157.854,157.883,157.865,157.892,157.864,157.892,157.846,157.875,157.844,157.874,EUR_JPY,0.018,157.865733,157.880844,157.850623,0.01036,1,0.0,0,0.0,0.0,157.883
6857,2024-01-08T16:52:00.000000000Z,157.880,157.898,157.872,157.893,157.889,157.907,157.881,157.902,157.872,157.888,157.862,157.884,EUR_JPY,0.018,157.867333,157.887708,157.846959,0.01540,1,1.0,0,0.0,0.0,157.893
6858,2024-01-08T16:53:00.000000000Z,157.897,157.905,157.888,157.893,157.906,157.915,157.896,157.903,157.888,157.895,157.877,157.883,EUR_JPY,0.020,157.871367,157.896340,157.846394,0.01298,0,1.0,0,0.0,0.0,157.893
6859,2024-01-08T16:54:00.000000000Z,157.897,157.920,157.892,157.919,157.907,157.930,157.901,157.929,157.887,157.911,157.881,157.909,EUR_JPY,0.020,157.877000,157.908991,157.845009,0.02520,1,0.0,0,0.0,0.0,157.919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6930,2024-01-08T18:05:00.000000000Z,157.868,157.876,157.858,157.872,157.876,157.884,157.867,157.880,157.859,157.868,157.850,157.863,EUR_JPY,0.017,157.861100,157.886916,157.835284,0.00654,0,0.0,0,0.0,0.0,157.872
6931,2024-01-08T18:06:00.000000000Z,157.872,157.884,157.868,157.884,157.880,157.894,157.876,157.894,157.865,157.875,157.861,157.875,EUR_JPY,0.019,157.861167,157.887179,157.835154,0.01370,0,0.0,0,0.0,0.0,157.884
6932,2024-01-08T18:07:00.000000000Z,157.886,157.905,157.886,157.901,157.894,157.914,157.894,157.910,157.877,157.896,157.877,157.892,EUR_JPY,0.018,157.863367,157.897239,157.829495,0.02258,1,0.0,0,0.0,0.0,157.901
6933,2024-01-08T18:08:00.000000000Z,157.902,157.910,157.900,157.909,157.910,157.919,157.908,157.918,157.893,157.901,157.892,157.900,EUR_JPY,0.018,157.867733,157.911125,157.824342,0.02476,0,1.0,0,0.0,0.0,157.909


In [193]:
import plotly
import plotly.graph_objects as go

In [194]:

# Create the candlestick chart
fig = go.Figure(data=[go.Candlestick(
    x=df_pl.time, #['timestamp'],
    open=df_pl['mid_o'],
    high=df_pl['mid_h'],
    low=df_pl['mid_l'],
    close=df_pl['mid_c']
)])
fig.add_trace(go.Scatter(
    x=df_pl.time,
    y=df_pl['BB_UP'],
    mode='lines',
    line=dict(color='blue', width=2),
    name='BB_UP Moving Average'
))

In [195]:
# Add custom color for a specific candlestick
# df_pl[df_pl['Date'] == custom_color_candle]
fig.add_trace(go.Candlestick(
    x=df_an.iloc[index:index+1]['time'],
    open=df_an.iloc[index:index+1]['mid_o'],
    high=df_an.iloc[index:index+1]['mid_h'],
    low=df_an.iloc[index:index+1]['mid_l'],
    close=df_an.iloc[index:index+1]['mid_c'],
    increasing=dict(line=dict(color='orange'), fillcolor='orange'),
    decreasing=dict(line=dict(color='orange'), fillcolor='orange')
))


fig.show()